In [15]:
df = pd.read_csv("Finland.csv")

In [16]:
df = df.iloc[:, 3:]
df

,Datetime (Local),Price (EUR/MWhe)
0,2015-01-01 02:00:00,23.37
1,2015-01-01 03:00:00,19.33
2,2015-01-01 04:00:00,17.66
3,2015-01-01 05:00:00,17.53
4,2015-01-01 06:00:00,18.07
...,...,...
89889,2025-04-03 19:00:00,-2.12
89890,2025-04-03 20:00:00,-2.02
89891,2025-04-03 21:00:00,-2.03
89892,2025-04-03 22:00:00,-2.42


In [17]:
df.set_index("Datetime (Local)", inplace=True)
df.head(5)

,Price (EUR/MWhe)
Datetime (Local),
2015-01-01 02:00:00,23.37
2015-01-01 03:00:00,19.33
2015-01-01 04:00:00,17.66
2015-01-01 05:00:00,17.53
2015-01-01 06:00:00,18.07


In [18]:
df = df.rename(columns={
    df.columns[0]: "Price"        # renomme dynamiquement la 2e colonne
})

df

,Price
Datetime (Local),
2015-01-01 02:00:00,23.37
2015-01-01 03:00:00,19.33
2015-01-01 04:00:00,17.66
2015-01-01 05:00:00,17.53
2015-01-01 06:00:00,18.07
...,...
2025-04-03 19:00:00,-2.12
2025-04-03 20:00:00,-2.02
2025-04-03 21:00:00,-2.03


In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Paramètres
sequence_length = 168  # 1 semaine d'historique (168 heures)
horizon = 168          # Prédire 1 semaine (168 heures) dans le futur


# 2. Normalisation
scaler = MinMaxScaler()
df['Price_norm'] = scaler.fit_transform(df[['Price']])

# 3. Création des séquences pour le LSTM
X, y = [], []
for i in range(sequence_length, len(df) - horizon):
    X.append(df['Price_norm'].iloc[i - sequence_length:i].values.reshape(sequence_length, 1))
    y.append(df['Price_norm'].iloc[i:i + horizon].values)
    
X = np.array(X)
y = np.array(y)

print("Forme de X :", X.shape)  # (n_samples, 168, 1)
print("Forme de y :", y.shape)  # (n_samples, 168)

# 4. Découpage train/test (respect de l'ordre chronologique)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# 5. Construction d'un modèle LSTM avancé
model = Sequential()

# Couche 1 : Bidirectional LSTM avec retour des séquences
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(sequence_length, 1)))
model.add(Dropout(0.2))

# Couche 2 : Une seconde couche Bidirectional LSTM (sans retour de séquence)
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.2))

# Couche Dense intermédiaire pour enrichir la représentation
model.add(Dense(128, activation='relu'))

# Couche de sortie : Prédiction multi-sortie (horizon de 168 heures)
model.add(Dense(horizon))

model.compile(optimizer='adam', loss='mse')
model.summary()

# 6. Entraînement avec EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)

# 7. Prédiction sur le jeu de test
y_pred = model.predict(X_test)

# 8. Inversion de la normalisation pour obtenir des valeurs réelles
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

# 9. Visualisation de la prédiction sur le 1er échantillon du test
plt.figure(figsize=(12, 6))
plt.plot(y_test_inv[0], label='Prix Réel')
plt.plot(y_pred_inv[0], label='Prix Prévu')
plt.title("Prédiction sur 1 semaine (168h) avec modèle avancé LSTM")
plt.xlabel("Heures dans le futur")
plt.ylabel("Prix (€/MWh)")
plt.legend()
plt.tight_layout()
plt.show()


Forme de X : (89558, 168, 1)
Forme de y : (89558, 168)


/Users/ethanbareille/miniconda3/envs/ml_env/lib/python3.11/site-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)   │ (None, 168, 256)       │       133,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 168, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 168)            │        21,672 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 335,656 (1.28 MB)

 Trainable params: 335,656 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 293s 130ms/step - loss: 0.0018 - val_loss: 8.4403e-04
Epoch 2/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 292s 130ms/step - loss: 4.1871e-04 - val_loss: 8.3886e-04
Epoch 3/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 298s 133ms/step - loss: 4.7886e-04 - val_loss: 0.0012
Epoch 4/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 299s 133ms/step - loss: 8.5148e-04 - val_loss: 9.3728e-04
Epoch 5/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 298s 133ms/step - loss: 7.8297e-04 - val_loss: 0.0010
Epoch 6/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 298s 133ms/step - loss: 0.0011 - val_loss: 8.8970e-04
Epoch 7/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 297s 133ms/step - loss: 7.2964e-04 - val_loss: 0.0012
Epoch 8/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 298s 133ms/step - loss: 6.6922e-04 - val_loss: 0.0020
Epoch 9/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 285s 127ms/step - loss: 0.0013 - val_loss: 9.6770e-04
Epoch 10/200
2239/2239 ━━━━━━━━━━━━━━━━━━━━ 300s 134ms/step - loss: 6.2215e-04 - val_loss: 0.0010
Epoch 11/200
2239/223